In [1]:
def calculateE2Edelay(dataO, cloudsim, output):

    #read cloudsim delay file
    # format: time, VM, delay, description
    print(cloudsim)

    dataCS = pd.read_csv(cloudsim + ".csv", sep=' ', header=0,)

    
    # rename Time to time, and VM to car, to align with omnet dataframe
    # rename delay to csd to distinguish from omnet
    dataCS=dataCS.rename(columns={"Time": "time", "VM": "car", "delay":"csd"})
    

    # concatenate cloudsim dataframe with omnet dataframe

    dataCars = pd.concat([dataO, dataCS], join = 'outer', ignore_index = True)

    # sort by time
    dataCars = dataCars.sort_values(by=['time'])
    dataCars = dataCars.reset_index(drop=True)
    dataCars = dataCars.set_index('car', append=True)

    
    # create subsets by car / VM
    
    dataCars["csd"] = dataCars.groupby('car')['csd'].transform(lambda x: x.ffill())
#    print(dataCars)   
    
#    dataCars = dataCars.fillna(999)
# print NaN info
    print(dataCars.isna().sum()/len(dataCars))
# drop NAN values
    dataCars.dropna()
#    print(dataCars)   

# all delay information should be in ms        
    dataCars['e2e'] = dataCars['delay']*1000 + dataCars['csd']
    dataCars['delay'] = dataCars['delay']*1000
    
    # only keep rows with omnet description
    filtered = dataCars[dataCars['description'] == 'omnet']
    filtered = filtered.reset_index()
    filtered = filtered.drop(columns=['description','level_0'])
    filtered = filtered.round(2)

        
    filtered.to_csv(output + '.csv', index=False)
    
#    print(filtered)

# create bins with size of 10 seconds and calculate average e2e delay per car per bin

    maxTime = filtered['time'].max()
# bin size = 10 seconds
    cuts = pd.cut(filtered['time'], np.arange(1,maxTime+1,10))

    binned = filtered.groupby(['car', cuts]).e2e.mean().reset_index(name='avg_e2e')

# drop the bins with no information
    binned = binned.dropna()
    binned = binned.round(2)

    binned.to_csv(output + '_bin.csv', index=False)


#print(binned)



In [2]:
# measuring execution time
%load_ext autotime

# cycle through all simulations and call function to calculate e2e delay

import vaex as vx
import numpy as np
import pandas as pd


text = "./VMs_network_delay/VMs_network_delay_"
outFile = "e2eDelay_"
cars = np.array([4951, 4955, 4956, 5734, 5740, 5749, 6910, 6915, 6923, 8600, 8620, 8635])
hosts = np.array([45, 63, 81, 99, 117])
hostType = np.array([0, 1])
midtext = "_sc_sc-mad_"
services = np.array([5074, 5013, 4256, 5808, 4966, 5858, 6042, 6187, 7004, 7455, 7463, 7486])
ind = 0

#cars = np.array([4928])
#hosts = np.array([45])
#hostType = np.array([0])
#services = np.array([1800])



for i in cars:
    # read omnet delay file
    # format: car, time, delay
    dataO = pd.read_csv(str(i) + '.csv', sep=',', header=0)
    # add description column to omnet dataframe
    dataO['description'] = 'omnet'


    for h in hosts:
        for t in hostType:
            cloudsim = text + str(h) + '_' + str(t) + midtext + str(services[ind]) + '_' + str(i)
            output = outFile + str(h) + '_' + str(t) + midtext + str(services[ind]) + '_' + str(i)
            calculateE2Edelay(dataO, cloudsim, output)
    
    ind = ind + 1


./VMs_network_delay/VMs_network_delay_45_0_sc_sc-mad_5074_4951


INFO:MainThread:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:MainThread:numexpr.utils:NumExpr defaulting to 8 threads.


time           0.000000
delay          0.014063
description    0.000000
csd            0.131117
dtype: float64
./VMs_network_delay/VMs_network_delay_45_1_sc_sc-mad_5074_4951
time           0.000000
delay          0.013086
description    0.000000
csd            0.027897
dtype: float64
./VMs_network_delay/VMs_network_delay_63_0_sc_sc-mad_5074_4951
time           0.000000
delay          0.013767
description    0.000000
csd            0.043949
dtype: float64
./VMs_network_delay/VMs_network_delay_63_1_sc_sc-mad_5074_4951
time           0.000000
delay          0.010786
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_81_0_sc_sc-mad_5074_4951
time           0.000000
delay          0.012098
description    0.000000
csd            0.000200
dtype: float64
./VMs_network_delay/VMs_network_delay_81_1_sc_sc-mad_5074_4951
time           0.000000
delay          0.008522
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay

./VMs_network_delay/VMs_network_delay_117_0_sc_sc-mad_4966_5740
time           0.000000
delay          0.002772
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_117_1_sc_sc-mad_4966_5740
time           0.000000
delay          0.002772
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_45_0_sc_sc-mad_5858_5749
time           0.000000
delay          0.013985
description    0.000000
csd            0.112424
dtype: float64
./VMs_network_delay/VMs_network_delay_45_1_sc_sc-mad_5858_5749
time           0.000000
delay          0.013115
description    0.000000
csd            0.023099
dtype: float64
./VMs_network_delay/VMs_network_delay_63_0_sc_sc-mad_5858_5749
time           0.000000
delay          0.013413
description    0.000000
csd            0.040548
dtype: float64
./VMs_network_delay/VMs_network_delay_63_1_sc_sc-mad_5858_5749
time           0.000000
delay          0.010601
description    0.

time           0.000000
delay          0.002796
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_99_0_sc_sc-mad_7455_8600
time           0.000000
delay          0.002911
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_99_1_sc_sc-mad_7455_8600
time           0.00000
delay          0.00279
description    0.00000
csd            0.00000
dtype: float64
./VMs_network_delay/VMs_network_delay_117_0_sc_sc-mad_7455_8600
time           0.000000
delay          0.002823
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_117_1_sc_sc-mad_7455_8600
time           0.000000
delay          0.002787
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/VMs_network_delay_45_0_sc_sc-mad_7463_8620
time           0.000000
delay          0.008381
description    0.000000
csd            0.000000
dtype: float64
./VMs_network_delay/V